# Predicción de Churn en **Beta Bank**

---

## 1) **Business Understanding**

### 1.1. Contexto
Los clientes de **Beta Bank** están abandonando el banco gradualmente. Captar clientes nuevos es más costoso que retener a los actuales. Necesitamos **identificar a tiempo** a quienes tienen alta probabilidad de churn para priorizar acciones de retención.

### 1.2. Problema de negocio
Crear un sistema de **clasificación binaria** que prediga si un cliente **se irá** pronto.

- **Variable objetivo**: `Exited` (1 = se fue, 0 = se queda)

### 1.3. Métricas y criterios de éxito
- **Métrica principal**: F1 en el **conjunto de prueba** $\rightarrow$ **F1 ≥ 0.70**.  
- **Métrica secundaria**: AUC-ROC (comparar y discutir con F1).  
- Justificación:
  - **F1** equilibra *precision* y *recall* ante **desequilibrio** de clases.  
  - **AUC-ROC** evalúa la capacidad de ranking del modelo.

**Recordatorio de fórmulas**:  
- F1 (macro-definición binaria):
  $$
  F1 = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
  $$
- Donde:
  - $\text{Precision} = \frac{TP}{TP+FP}$  
  - $\text{Recall} = \frac{TP}{TP+FN}$

---

## 2) **Data Understanding**

### 2.1. Fuente y acceso
- **Ruta**: `/datasets/Churn.csv`

### 2.2. Diccionario de datos (resumen)
**Características**
- `RowNumber`: índice de fila  
- `CustomerId`: identificador único  
- `Surname`: apellido  
- `CreditScore`: score de crédito  
- `Geography`: país de residencia  
- `Gender`: sexo  
- `Age`: edad  
- `Tenure`: antigüedad del depósito (años)  
- `Balance`: saldo  
- `NumOfProducts`: número de productos bancarios  
- `HasCrCard`: tiene tarjeta (1/0)  
- `IsActiveMember`: cliente activo (1/0)  
- `EstimatedSalary`: salario estimado

**Objetivo**
- `Exited`: churn (1/0)

### 2.3. Tareas de comprensión de datos (sin código)
- [ ] Verifica cantidad de filas y columnas, **tipos de datos**, valores faltantes, duplicados.  
- [ ] Revisa **estadísticos descriptivos** de variables numéricas y cardinalidad de categóricas.  
- [ ] Analiza **distribuciones** y posibles **outliers** (Balance, Age, EstimatedSalary).  
- [ ] Evalúa **correlaciones** y relaciones relevantes con `Exited`.  
- [ ] **Equilibrio de clases**: calcula la proporción de `Exited=1` vs `Exited=0`.

> **Requisito** (instrucción del proyecto):  
> Entrena **un primer modelo base sin tratar el desequilibrio** y registra hallazgos (qué tan mal/bien va F1 y AUC-ROC).

---

## 3) **Data Preparation**

> Objetivo: transformar datos crudos en un conjunto listo para modelado, **sin fuga de información**.

### 3.1. Limpieza y validaciones
- [ ] Tratamiento de **valores faltantes** (numerical / categorical).  
- [ ] Detección y manejo de **duplicados**.  
- [ ] Chequeo de **rangos válidos** (edad positiva, saldos no negativos, etc.).  
- [ ] Decisiones documentadas: imputación, exclusión o transformación.

### 3.2. Ingeniería de características (opcional, sugerida)
- [ ] Variables derivadas:
  - *Ratios* (p.ej., `Balance/EstimatedSalary`),  
  - *Binning* de `Age` o `Tenure`,  
  - Interacciones si aportan.  
- [ ] Normalización/Estandarización para modelos sensibles a escala.

### 3.3. Codificación de variables categóricas
- [ ] **`Geography`, `Gender`**: *One-Hot Encoding* o *Target Encoding* (justifica elección).  
- Evita **leakage**: ajusta codificadores **solo con entrenamiento**.

### 3.4. División de datos
- [ ] **Stratified** *train/validation/test* (p. ej., 60/20/20 o 70/15/15).  
- Mantén la **misma proporción de clases** en cada conjunto.  
- Fija `random_state` y documenta el **protocolo de evaluación**.

> **Nota**: Todo el preprocesamiento debe ir dentro de **pipelines** (conceptualmente) para evitar fugas entre train/valid/test.

---

## 4) **Modeling**

### 4.1. Línea base (baseline)
- [ ] Entrena un modelo sencillo **sin tratar desequilibrio** (p. ej., Regresión Logística / Árbol simple).  
- [ ] Registra F1 y AUC-ROC en **validación**.  
- [ ] Usa esta línea base como referencia.

### 4.2. Enfoques para el **desequilibrio de clases** (usa **al menos dos**)
- **Ponderación**: `class_weight='balanced'` en modelos compatibles.  
- **Re-muestreo**:
  - **Oversampling** (p. ej., **SMOTE** / Random OverSampling).  
  - **Undersampling** (Random UnderSampling).  
- **Ajuste de umbral** (*threshold moving*):
  - Selecciona el umbral de probabilidad que **maximiza F1** en validación.  
- (Opcional) **Enfoques en el espacio de decisiones**: ensembles focalizados, calibración de probabilidades.

> Sugerencia: compara **al menos 2** técnicas (p. ej., *class_weight* vs **SMOTE**).

### 4.3. Modelos a evaluar (requeridos)
Entrena y compara (en *validation*) varios algoritmos:
- **Logistic Regression**
- **Tree** (e.g., `DecisionTreeClassifier`)  
- **Random Forest** (`RandomForestClassifier`)  

> Documenta: hiperparámetros, técnica de re-muestreo/ponderación usada y resultados.

### 4.4. Búsqueda de hiperparámetros
- [ ] Usa validación cruzada estratificada en **conjunto de entrenamiento** y evalúa en **validación**.  
- [ ] Prueba **GridSearch** (o RandomizedSearch si es más eficiente).  
- [ ] Escoge la **mejor combinación** por **F1** (criterio primario).  
- [ ] **Ajusta umbral** del mejor modelo según la curva **Precision-Recall** para maximizar F1.

### 4.5. Métricas y artefactos a registrar (en validación)
- [ ] **F1** y **AUC-ROC**.  
- [ ] Matriz de confusión (TP, FP, TN, FN).  
- [ ] Curvas ROC y Precision-Recall.  
- [ ] Importancia de variables.
- [ ] Umbral óptimo elegido y motivación.

---

## 5) **Evaluation**

### 5.1. Prueba final (Test)
- [ ] Reentrena (si corresponde) con **train+valid** y evalúa en **test**.  
- [ ] Reporta **F1 (debe ser ≥ 0.70)** y **AUC-ROC** en **test**.  
- [ ] Compara y discute F1 vs AUC-ROC:
  - ¿El modelo ranquea bien (AUC-ROC alto) pero falla en el umbral para F1?  
  - ¿El ajuste de umbral mejoró F1 sin degradar demasiado AUC-ROC?

### 5.2. Análisis de error y robustez
- [ ] Segmenta desempeño por **Geography**, **Gender**, **Age groups**, **IsActiveMember**.  
- [ ] Identifica **sesgos** o grupos con alto **FN** (falsos negativos) y propone mitigaciones.  
- [ ] Chequea estabilidad del modelo (variación por semillas / folds).

### 5.3. Conclusiones
- [ ] Resume: **mejor modelo**, técnica(s) anti-desequilibrio, umbral final, y métricas clave en **test**.  
- [ ] Discute **riesgos** (leakage, sobreajuste, dependencia de re-muestreo).

---

## 6) **Deployment**

Exportar el modelo como binario

---

## 📎 Anexos

### A) Descripción completa de los datos
- `RowNumber` (int) — índice de fila  
- `CustomerId` (int/str) — id único  
- `Surname` (str) — apellido  
- `CreditScore` (num) — score de crédito  
- `Geography` (cat) — país  
- `Gender` (cat) — sexo  
- `Age` (num) — edad  
- `Tenure` (num) — años de antigüedad del depósito  
- `Balance` (num) — saldo  
- `NumOfProducts` (num) — nº de productos  
- `HasCrCard` (bin) — tarjeta (1/0)  
- `IsActiveMember` (bin) — activo (1/0)  
- `EstimatedSalary` (num) — salario estimado  
- **Objetivo**: `Exited` (bin)

### B) Posibles fugas de información (evítalas)
- Usar **estadísticos globales** calculados con todo el dataset para imputación/encoding antes del split.  
- Aplicar **SMOTE/undersampling/oversampling/standardization** fuera del **pipeline** (deben ajustarse usando **solo** train).  
- Selección de variables basada en toda la data (hazla con *cross-validation* o dentro de pipeline).

### C) Estructura sugerida del *repo/cuaderno*
- **Secciones** de este documento como títulos de celdas Markdown.  
- **Resultados** y **gráficos** colocados al final de cada etapa con una mini-conclusión.

---

## Rubrica de Evaluación (Checklist para revisión)

- [ ] Preparación de datos clara y correcta (todos los tipos procesados).  
- [ ] Explicación de pasos de preprocesamiento y decisiones justificadas.  
- [ ] Investigación del **equilibrio de clases** y baseline sin corrección.  
- [ ] Al menos **dos técnicas** para corregir desequilibrio (**class_weight**, **SMOTE/over**, **under**, **over**,**threshold**).  
- [ ] División **estratificada** y protocolo de validación bien definido.  
- [ ] Entrenamiento, validación y prueba **correctos y reproducibles**.  
- [ ] **F1 en test ≥ 0.70** reportado junto a **AUC-ROC**.  
- [ ] Comparación y análisis crítico **F1 vs AUC-ROC**.  
- [ ] Estructura clara, limpieza y orden del cuaderno.


# ROC–AUC (enfoque práctico)

## Qué mide
- **ROC**: curva de pares $(\mathrm{FPR}(\tau), \mathrm{TPR}(\tau))$ al variar el umbral $\tau$.
  - $\mathrm{TPR}=\frac{TP}{TP+FN}$ (recall),  
    $\mathrm{FPR}=\frac{FP}{FP+TN}$.
- **AUC**: área bajo la ROC. Interpretación clave:
  $$
  \mathrm{AUC}=\mathbb{P}(S_1>S_0)+\tfrac{1}{2}\mathbb{P}(S_1=S_0)
  $$
  Probabilidad de que un positivo tenga **mayor score** que un negativo (mide **ranking**, no calibración).

## Por qué importa
- Invariante a transformaciones monótonas del score y **poco sensible a la prevalencia** de la clase.
- Resume el rendimiento para **todos los umbrales**; útil para comparar modelos sin fijar $\tau$.

## Cuándo usar (y cuándo no)
- Para **elegir modelo** por capacidad de ranking global.
- En **datasets muy desbalanceados**, AUC puede verse optimista. Complementa con **PR/AUC-PR** y **F1**.

## Selección de umbral (lo operativo)
- AUC no da el umbral. Elige $\tau$ según objetivo:
  - **Max F1** si balanceas precisión y recall.
  - **Costo mínimo** si tienes matriz de costos:
    $$
    \mathbb{E}[C]=C_{FP}\cdot FP(\tau)+C_{FN}\cdot FN(\tau)
    $$
- Usa curvas **Precision–Recall** y la **matriz de confusión** para justificar la decisión.

## Buenas prácticas
1. Separar **train/valid/test** y evitar **leakage** en todo el preprocesamiento.
2. Comparar modelos con **AUC (± IC)** y reportar también **F1**, **precision**, **recall**.
3. Analizar **PR/AUC-PR** si hay fuerte desequilibrio.
4. Reportar el **umbral final** y por qué lo elegiste.
5. Considerar **calibración** (e.g., Platt/Isotónica) si usarás **probabilidades** en negocio.

## Mini–resumen para decidir rápido
- **Escoger modelo**: mira **AUC** (y AUC-PR si desbalanceado).
- **Operar**: fija **umbral** para tu objetivo (F1 o costo).
- **Monitorear**: revisa F1/ROC–AUC periódicamente y reentrena ante *drift*.

**Reglas de pulgar**  
- AUC ≈ 0.5: aleatorio; 0.7–0.8: aceptable; 0.8–0.9: bueno; >0.9: excelente (verifica que no haya fuga).
